In [ ]:
import zipfile
import os
import re
import shutil
import nltk
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import openai
import time
import random

# PART 1: Data Loading and Preprocessing 

# Load environment variables
load_dotenv()

# NLTK Downloads (you might need to uncomment these if you haven't downloaded them)
# nltk.download('vader_lexicon')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

def preprocess_text(text):
    """Preprocesses the given text (handles HTML, CSS, and plain text)."""
    # Remove HTML/CSS tags using Beautiful Soup with lxml parser
    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text(separator=" ", strip=True)

    # Remove punctuation and convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()

    return text

def load_and_preprocess_data(zip_file_path):
    """Loads and preprocesses data from the zip file."""
    data = {}
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall('temp_data')

    inner_folder = os.listdir('temp_data')[0]
    print(f"Inner folder found: {inner_folder}")

    for filename in os.listdir(os.path.join('temp_data', inner_folder)):
        if filename.endswith(".txt"):
            file_type = filename.split("_")[0]
            try:
                with open(os.path.join('temp_data', inner_folder, filename), 'r', encoding='utf-8') as file:
                    text = file.read()
                    processed_text = preprocess_text(text)
                    if file_type not in data:
                        data[file_type] = []
                    data[file_type].append(processed_text)

                    print(f"  Loaded file: {filename}")
                    print(f"  Processed text (first 100 chars): {processed_text[:100]}...")

            except UnicodeDecodeError:
                print(f"UnicodeDecodeError while reading {filename}. Skipping this file.")

    try:
        shutil.rmtree('temp_data')
    except PermissionError:
        print("PermissionError: Could not delete 'temp_data' directory. Please close any open files and try again.")
    except OSError as e:
        print(f"Error deleting 'temp_data' directory: {e}")

    print(f"  Loaded data: {data}")
    return data

# --- PART 2: Topic Extraction ---

openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_topics(text, delay_seconds):
    """
    Extracts topics from the given text using the ChatGPT API,
    guided by key themes and seed topics.
    """
    print(f"Extracting topics from text: {text[:50]}...")

    key_themes = [
        "user experience", "technical issues", "personalization",
        "accessibility", "peak hours", "customer support"
    ]

    seed_topics = [
        "teen therapy", "group therapy", "session notes", "teletherapy audio",
        "crisis response", "user onboarding", "video quality",
        "appointment scheduling", "therapy companion app", "payment system",
        "cultural matching", "patient engagement", "therapist matching"
    ]

    prompt = f"""
    Extract the key themes and topics discussed in the following text,
    focusing on aspects like {", ".join(key_themes)} and
    considering the context of {", ".join(seed_topics)}:

    Text: \"\"\"{text}\"\"\"
    """

    try:
        client = openai.OpenAI()
        time.sleep(delay_seconds)  # Delay before each API call
        for attempt in range(5):  # Retry up to 5 times
            try:
                response = client.chat.completions.create(
                    model="gpt-3.5-turbo",  # Or a cheaper model if available
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant that extracts key topics from text."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.7,
                    max_tokens=150,
                    top_p=1.0,
                    frequency_penalty=0.0,
                    presence_penalty=0.0,
                )
                raw_topics = response.choices[0].message.content.split('\n')
                topics = [topic.strip() for topic in raw_topics if topic.strip()]
                print(f"  Extracted topics from OpenAI: {topics}")
                return topics

            except openai.RateLimitError as e:
                wait_time = (4 ** attempt) + random.uniform(0, 5)  # Aggressive backoff
                print(f"Rate limit exceeded, waiting for {wait_time:.2f} seconds...")
                time.sleep(wait_time)

        else:
            print("Failed after multiple retries.")
            return []

    except Exception as e:
        print(f"Error in extract_topics: {e}")
        return []

# --- PART 3: Sentiment Analysis and Data Point Extraction ---

def analyze_sentiment(text):
    """Performs sentiment analysis on the given text."""
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(text)['compound']
    if sentiment_score >= 0.05:
        return "positive"
    elif sentiment_score <= -0.05:
        return "negative"
    else:
        return "neutral"

def extract_data_points(text):
    """Extracts relevant data points (percentages, numbers, dates) from the text."""
    try:
        data_points = []

        # Improved date pattern to capture complete dates
        date_pattern = r"\b\d{4}-\d{2}-\d{2}\b"
        data_points.extend(re.findall(date_pattern, text))

        # Percentage pattern (no change needed)
        percentage_pattern = r"(\d+\.?\d*)%"
        data_points.extend(re.findall(percentage_pattern, text))

        # Number pattern (modified to avoid capturing parts of dates)
        number_pattern = r"(?<!\d-)(?<!\d)\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\b(?!\d)"
        data_points.extend(re.findall(number_pattern, text))

        # Remove duplicates by converting to a set and back to a list
        data_points = list(set(data_points))

        return data_points
    except Exception as e:
        print(f"Error in extract_data_points: {e}")
        return []

# --- PART 4: Story Generation ---
def generate_summary_with_chatgpt(text, topics, data_points, references):
    """
    Generates a summary using the ChatGPT API, focusing on key themes and data points.
    """
    try:
        client = openai.OpenAI()

        prompt = f"""
        Generate a concise summary for a product story, based on the following information:

        File Content (Text): \"\"\"{text}\"\"\"

        Extracted Topics: {", ".join(topics)}

        Data Points: {", ".join(data_points)}

        References: {", ".join(references)}

        The summary should highlight key insights, user experiences, and relevant data points.
        Focus on clarity and conciseness.
        """

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that generates product story summaries."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=250,  # Adjust as needed for summary length
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
        )

        summary = response.choices[0].message.content.strip()
        return summary

    except Exception as e:
        print(f"Error in generate_summary_with_chatgpt: {e}")
        return None

def generate_story(story_type, topics, sentiment, data_points, references, text):
    """
    Generates a product story based on the given information and detailed templates.
    """
    print(f"      Generating story of type: {story_type}")

    # Define templates for different story types with additional sections
    templates = {
        "Concern": {
            "headline_template": "[Problem/Issue] is Negatively Impacting [User/Area]",
            "summary_template": "Analysis of {references} reveals concerns regarding {topics}. {data_points} indicate a negative impact on {user_group}.",
            "impact_template": "This issue has led to {impact}.",
            "user_perspective_template": "Users have expressed {user_perspectives} about this problem.",
            "recommendation_template": "To address this concern, it is recommended to {recommendations}.",
        },
        "Win": {
            "headline_template": "[Positive Outcome/Achievement] in [Area/Feature]",
            "summary_template": "{data_points} highlight the success of {feature} in {references}. User feedback indicates {positive_feedback}.",
            "contributing_factors_template": "This success can be attributed to {factors}.",
            "impact_template": "This positive outcome has resulted in {impact}.",
            "user_perspective_template": "Users have expressed {user_perspectives} about this feature.",
            "recommendation_template": "To maintain and further enhance this success, it is recommended to {recommendations}."
        },
        "Insight": {
            "headline_template": "Key Insight: [New Understanding/Discovery] in [Area/Feature]",
            "summary_template": "Analysis of {references} reveals a key insight: {insight}. This has implications for {area}.",
            "evidence_template": "This insight is supported by the following evidence: {evidence}.",
            "implication_template": "This insight suggests that {implication}.",
            "recommendation_template": "We recommend {recommendations} to leverage this insight."
        },
        "Opportunity": {
            "headline_template": "Opportunity to [Improve/Innovate] in [Area/Feature]",
            "summary_template": "{references} highlight an opportunity to {action} in {area}. This could lead to {benefits}.",
            "potential_template": "This opportunity has the potential to {potential}.",
            "recommendation_template": "To capitalize on this opportunity, it is recommended to {recommendations}."
        }
    }

    template = templates.get(story_type)
    if not template:
        return "Error: Invalid story type."

    # --- Input for Placeholders based on the analysis ---

    if story_type == "Concern":
        impact = "increased frustration and decreased engagement among users"
        user_perspectives = "'The app is slow and buggy,' and 'I'm having trouble finding the information I need'"
        recommendations = "optimize the app's performance, improve navigation, and provide clearer instructions"
    elif story_type == "Win":
        factors = "the user-friendly interface, personalized content, and effective communication tools"
        impact = "increased user satisfaction, higher engagement rates, and improved clinical outcomes"
        user_perspectives = "'The new onboarding process is much smoother and more intuitive,' and 'I feel like the app really understands my needs now'"
        recommendations = "continue gathering user feedback, expand the platform's reach, and explore new features based on user needs"
    elif story_type == "Insight":
        evidence = "Data from product_metrics_3.txt shows an 85% client retention rate and a 92% session completion rate for teen therapy."
        implication = "Teenagers are effectively engaging with the platform and finding value in the services offered."
        recommendations = "Continue to monitor these metrics and gather feedback to ensure ongoing satisfaction and engagement among teen users."
    elif story_type == "Opportunity":
        potential = "significantly enhance the user experience, improve accessibility, and potentially increase user engagement and retention."
        recommendations = "conduct user research to gather more specific feedback, explore potential design solutions, and prioritize development based on user needs and impact."

    # Generate the story using the template and extracted information
    story = f"""
    ## {template['headline_template'].format(Problem=topics[0], User='users', Area=topics[0])}

    {template['summary_template'].format(references=", ".join(references), topics=", ".join(topics), data_points=", ".join(data_points), user_group="users", feature=topics[0], insight=topics[0], area=topics[0], action="improve", benefits="better user experience")}

    {template.get('impact_template', '').format(impact=impact)}
    {template.get('user_perspective_template', '').format(user_perspectives=user_perspectives)}
    {template.get('contributing_factors_template', '').format(factors=factors)}
    {template.get('evidence_template', '').format(evidence=evidence)}
    {template.get('implication_template', '').format(implication=implication)}
    {template.get('potential_template', '').format(potential=potential)}
    
    {template['recommendation_template'].format(recommendations=recommendations)}

    **References:**
    {", ".join(references)}
    """
    return story

def generate_product_stories(preprocessed_data):
    """
    Generates product stories for the given preprocessed data.
    """
    all_stories = []
    for file_type, files in preprocessed_data.items():
        if file_type in ["feedback_analysis", "interview_transcripts", "product_metrics", "product_intelligence_report", "user_journey"]:
            for i, text in enumerate(files):
                print(f"Processing file: {file_type}_{i+1}.txt")
                topics = extract_topics(text)
                sentiment = analyze_sentiment(text)
                data_points = extract_data_points(text)

                print(f"  Topics: {topics}")
                print(f"  Sentiment: {sentiment}")
                print(f"  Data points: {data_points}")

                # Story type classification logic 
                if sentiment == "negative":
                    if any(topic in ["technical issues", "connectivity problems", "errors", "delays", "sync issues", "audio quality", "video quality", "payment delays"] for topic in topics):
                        story_type = "Concern"
                    elif any(topic in ["accessibility issues", "limited availability", "mismatch", "long wait times"] for topic in topics):
                        story_type = "Concern"
                    #  We can add more rules here
                elif sentiment == "positive":
                    if any(topic in ["user satisfaction", "clinical improvement", "positive feedback", "high engagement", "effective communication", "successful implementation"] for topic in topics):
                        story_type = "Win"
                    elif any(topic in ["ease of use", "intuitive interface", "personalized experience", "high adoption rate"] for topic in topics):
                        story_type = "Win"
                    
                elif sentiment == "neutral":
                    if any(topic in ["user behavior", "new discovery", "insights", "trends", "peak hours", "user journey"] for topic in topics):
                        story_type = "Insight"
                  
                elif any(topic in ["opportunity", "potential", "enhancement", "innovation", "future development"] for topic in topics):
                    story_type = "Opportunity"
                else:
                    story_type = "Insight"  # Default to "Insight" if no other rules match

                story = generate_story(story_type, topics, sentiment, data_points, [f"{file_type}_{i+1}.txt"], text)
                print(f"  Generated story:\n{story}\n")

                all_stories.append(story)
    return all_stories

# --- INTEGRATION AND TESTING (Manual File Selection) ---

# Example Usage (Integrating all parts with manual file selection):
zip_file_path = r'C:\Users\Anshd\Downloads\data\take_home_data.zip'  # Your zip file path
preprocessed_data = load_and_preprocess_data(zip_file_path)

# VERY long delay (e.g.,here we use  60 seconds) - adjust as needed we do this as we are getting api rate limit exeeded error , so we provide a delay etween calls
delay_seconds = 60

# Select a single file type and file index to process
file_type_to_process = "design"  # Change this to process a different file type
file_index_to_process = 0

if file_type_to_process in preprocessed_data:
    text_list = preprocessed_data[file_type_to_process]

    if file_index_to_process < len(text_list):
        text = text_list[file_index_to_process]
        shortened_text = text[:500]  # Use only the first 500 characters for testing

        # Part 2: Topic Extraction
        topics = extract_topics(shortened_text, delay_seconds)
        print(f"  Topics extracted for {file_type_to_process}: {topics}")

        # Part 3: Sentiment Analysis and Data Point Extraction
        sentiment = analyze_sentiment(shortened_text)
        data_points = extract_data_points(shortened_text)

        print(f"  Sentiment for {file_type_to_process}: {sentiment}")
        print(f"  Data points extracted for {file_type_to_process}: {data_points}")

        # Part 4: Story Generation
        story = generate_story("Concern", topics, sentiment, data_points, [f"{file_type_to_process}_{file_index_to_process + 1}.txt"], shortened_text)
        print(f"  Generated story:\n{story}")

    else:
        print(f"  No file found at index {file_index_to_process} for {file_type_to_process}")
else:
    print(f"  File type '{file_type_to_process}' not found in preprocessed data.")

Inner folder found: take_home_data
  Loaded file: design_mockup_10.txt
  Processed text (first 100 chars): html teen therapy  sondermind home services about us contact teen therapy empowering teens supportin...
  Loaded file: design_mockup_100.txt
  Processed text (first 100 chars): html appointment scheduling  sondermind sondermind mental health platform appointments profile suppo...
  Loaded file: design_mockup_113.txt
  Processed text (first 100 chars): html therapy companion app sondermind therapy companion app dashboard resources support settings you...
  Loaded file: design_mockup_126.txt
  Processed text (first 100 chars): html sondermind payment system sondermind payment system manage your payments efficiently and secure...
  Loaded file: design_mockup_139.txt
  Processed text (first 100 chars): html cultural matching  sondermind cultural matching for better outcomes improve your mental health ...
  Loaded file: design_mockup_152.txt
  Processed text (first 100 chars): html pat

In [1]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
